In [1]:
%run ../ipynb_util_tars.py

In [2]:
%env WANDB_PROJECT=thesis

env: WANDB_PROJECT=thesis


In [3]:
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# LLaMA3 8B

## Load ZO_UP data

In [4]:
%run ../ipynb_load_data.py

{'SDG': ClassLabel(names=['1', '10', '11', '12', '13', '14', '15', '16', '17', '2', '3', '4', '5', '6', '7', '8', '9'], id=None), 'ABSTRACT': Value(dtype='string', id=None), 'id': Value(dtype='string', id=None), 'sdg_desc_short': Value(dtype='string', id=None), 'sdg_desc_long': Value(dtype='string', id=None)}
Example instance:	 {'SDG': 16, 'ABSTRACT': 'The first attempts to modernize simply replaced the single huge engine with a huge electric motor, changing little. The drive-shafts were replaced by wires, the huge steam engine by dozens of small motors. Factories spread out, there was natural light, and room to use ceiling-slung cranes. Workers had responsibility for their own machines, they needed better training and better pay. The electric motor was a wonderful invention, once we changed all the everyday details that surrounded it.', 'id': None, 'sdg_desc_short': None, 'sdg_desc_long': None}
Encoded (label2id) label:	 16
Decoded (id2label) label:	 9
9 16 16


In [5]:
import pprint
pprint.pprint(id2label)

{'1': 0,
 '10': 1,
 '11': 2,
 '12': 3,
 '13': 4,
 '14': 5,
 '15': 6,
 '16': 7,
 '17': 8,
 '2': 9,
 '3': 10,
 '4': 11,
 '5': 12,
 '6': 13,
 '7': 14,
 '8': 15,
 '9': 16}
{0: '1',
 1: '10',
 2: '11',
 3: '12',
 4: '13',
 5: '14',
 6: '15',
 7: '16',
 8: '17',
 9: '2',
 10: '3',
 11: '4',
 12: '5',
 13: '6',
 14: '7',
 15: '8',
 16: '9'}


## Model

In [6]:
from transformers.data import DataCollatorWithPadding
from transformers.models.llama import LlamaForSequenceClassification
from peft import LoraConfig, TaskType, get_peft_model

MODEL_NAME = "TinyLlama/TinyLlama_v1.1"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
model = LlamaForSequenceClassification.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    num_labels=len(labels),
    token=HF_TOKEN
).bfloat16()
model.config.pad_token_id = tokenizer.pad_token_id
# model.resize_token_embeddings(len(tokenizer))

encoded_dataset = dataset.map(
    preprocess_data(tokenizer), batched=True, remove_columns=dataset["train"].column_names
)
encoded_dataset.set_format("torch")

data_collator = DataCollatorWithPadding(tokenizer)

KeyboardInterrupt: 

In [ ]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import TrainingArguments, Trainer, EvalPrediction

BATCH_SIZE = 32
METRIC_NAME = "accuracy"

args = TrainingArguments(
    f"{CHECKPOINT_PATH}/{MODEL_NAME}",
    evaluation_strategy="epoch",
    logging_strategy="epoch",
    save_strategy="epoch",
    #eval_steps=50,
    #save_strategy="steps",
    #save_steps=50,
    #logging_steps=50,
    learning_rate=1e-4,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=5,
    load_best_model_at_end=True,
    save_total_limit=2,
    metric_for_best_model=METRIC_NAME,
    seed=SEED,
    report_to="wandb",
)

def compute_metrics(pred: EvalPrediction):
    labels = pred.label_ids
    accuracy = accuracy_score(labels, pred.predictions.argmax(-1))
    precision, recall, f1, _ = precision_recall_fscore_support(labels, pred.predictions.argmax(-1), average="weighted")
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


/home/user/dbielik/msc-thesis/venv/lib/python3.11/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: dvdblk (ngmi). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

{'eval_loss': 2.0351130962371826,
 'eval_accuracy': 0.7121771217712177,
 'eval_precision': 0.7142828266444502,
 'eval_recall': 0.7121771217712177,
 'eval_f1': 0.7060418491603037,
 'eval_runtime': 5.5771,
 'eval_samples_per_second': 48.592,
 'eval_steps_per_second': 1.614,
 'epoch': 5.0}

In [ ]:
from sklearn.metrics import classification_report

model.eval()

# manual evaluation to show classifcation_report
true_labels = []
logits = []

for batch in encoded_dataset["test"]:
    batch = {k: v.to(trainer.args.device).unsqueeze(0) for k, v in batch.items()}
    label = batch.pop("label")

    # Forward pass
    with torch.no_grad():
        out = model(**batch)

    true_labels.append(label.item())
    logits.extend(out.logits.tolist())

probabilites = torch.nn.functional.softmax(torch.tensor(logits), dim=-1)
pred_labels = torch.argmax(probabilites, dim=-1).tolist()

report = classification_report(true_labels, pred_labels, target_names=[f"SDG {id2label[i]}" for i in range(len(labels))])

In [ ]:
print(report)

              precision    recall  f1-score   support

       SDG 1       0.55      0.65      0.59        17
      SDG 10       0.38      0.35      0.36        17
      SDG 11       0.73      0.65      0.69        17
      SDG 12       0.64      0.53      0.58        17
      SDG 13       0.74      0.82      0.78        17
      SDG 14       1.00      0.94      0.97        17
      SDG 15       0.79      0.88      0.83        17
      SDG 16       0.75      0.53      0.62        17
      SDG 17       0.00      0.00      0.00         1
       SDG 2       0.64      0.88      0.74        16
       SDG 3       0.68      0.76      0.72        17
       SDG 4       0.79      0.88      0.83        17
       SDG 5       0.85      1.00      0.92        17
       SDG 6       0.75      0.88      0.81        17
       SDG 7       0.83      0.59      0.69        17
       SDG 8       0.64      0.44      0.52        16
       SDG 9       0.62      0.59      0.61        17

    accuracy              